In [18]:
import pandas as pd
import numpy as np
import os, sys
sys.path.insert(0, os.path.abspath('../..'))
import source.data_preparation as dp
import source.regression_models as modl
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
# Example


In [19]:
data = dp.prepare_data()

c:\Users\kilia\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [20]:
data

,price,d12,e12,ret,retx,AAA,BAA,lty,ltr,corpr,...,rdsp,rsvix,gpce,gip,tchi,house,avgcor,shtint,disag,equity_premium
date,,,,,,,,,,,,,,,,,,,,,
1871-01-01,4.44,0.260000,0.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1871-02-01,4.50,0.260000,0.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1871-03-01,4.61,0.260000,0.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1871-04-01,4.74,0.260000,0.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1871-05-01,4.86,0.260000,0.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-01,5648.40,72.925466,198.823333,0.024283,0.022898,0.0487,0.0560,0.0387,0.012792,0.015726,...,0.034581,NaN,NaN,NaN,1.055849,NaN,0.195877,-1.884540,2.504554,0.019204
2024-09-01,5762.48,73.400300,200.270000,0.022821,0.021707,0.0468,0.0542,0.0372,0.012023,0.017730,...,0.023585,NaN,NaN,NaN,1.055849,NaN,0.236572,-2.006363,2.263729,0.018572
2024-10-01,5705.45,73.877618,203.570000,-0.008690,-0.009523,0.0495,0.0563,0.0410,-0.023811,-0.024271,...,0.029564,NaN,NaN,NaN,1.055849,NaN,0.256120,-2.057976,2.846782,-0.012620


In [ ]:
#data = data[data.index < "2016-12-31"]

In [ ]:
# Pick the monthly variables you actually have in `data`:
monthly_vars = [
    "vp","impvar","vrp","lzrt","ogap","wtexas","sntm","ndrbL","skvw","tail",
    "fbm","dtoy","dtoat","ygap","rdsp","rsvix","tchi","avgcor","shtint",
    "disag","ntis","tby","lty","ltr","tms","dfy","dfr","infl"
]
monthly_vars = [v for v in monthly_vars if v in data.columns]

ranking = modl.rank_monthly_predictors(
    data,
    monthly_vars=monthly_vars,
    start_date="1927-01-01",
    start_oos="1965-01-01",
    lag=1,
    quiet=True,  # set False to see printed ranking
)

In [ ]:
from importlib import reload
reload(modl)
r2_o, y_true_o, y_pred_o, dates_o = modl.ols_oos(data, variables=("ogap",), start_oos="1965-01-01", mode="mean", ct_cutoff = True)
print(r2_o)
modl.plot_oos(
    y_true_o, y_pred_o, dates=dates_o,
    title=f"OLS(d/p) OOS (R²_OS={r2_o[0]:.4f})",
    ylabel="Monthly equity premium",
    mode = "mean"
)


In [ ]:
r2_c, y_true_c, y_pred_c, dates_c = modl.chronos_oos(data, start_oos="1965-01-01", quiet=False, ct_cutoff=True, freq='M', mode= "mean")

modl.plot_oos(
    y_true_c[1], y_pred_c[1], dates=dates_c[1],
    title=f"Chronos-Bolt OOS (R²_OS={r2_c[1]:.4f})",
    ylabel="Monthly equity premium", mode = "mean"
)


In [ ]:
vars_gw = ['d/p','d/y','e/p','tms','dfy','dfr','tbl','ltr','infl','svar','b/m','ntis','cay']
vars_core = ['d/p','e/p','tms','dfy','tbl','ltr','infl','svar','cay']

xgb_params = dict(
    n_estimators=1200, max_depth=2, learning_rate=0.01,
    subsample=0.7, colsample_bytree=0.7, reg_lambda=2.0,
    objective="reg:squarederror", random_state=42
)
# r2_te, y_true_te, y_pred_te, dates_te = modl.tree_ensemble_oos(data, vars_core, model_params=xgb_params, ct_cutoff=True)
# print("Tree ensemble OOS R²:", r2)
# # choose some variables you have
# vars_for_tree = ["tms","dfy","ntis"]

# modl.plot_oos(
#     y_true_te, y_pred_te, dates=dates_te,
#     title=f"Tree Ensemble OOS ({','.join(vars_for_tree)})  R²_OS={r2_te:.4f}",
#     ylabel="Monthly equity premium"
# )
r2_t, y_true_t, y_pred_t, dates_t = modl.tree_ensemble_oos(data, variables=vars_core, ct_cutoff=True, model_params=xgb_params, mode = "mean")
    
modl.plot_oos(
    y_true_t, y_pred_t, dates=dates_t,
    title=f"Tree Ensemble OOS (R²_OS={r2_t:.4f})",
    ylabel="Monthly equity premium",mode = "mean"
)

In [ ]:


r2_tf, y_true_tf, y_pred_tf, dates_tf = modl.timesfm_oos(
    data, start_oos="2008-09-01", min_context=120, max_context=512, ct_cutoff=True, quiet=False, freq="M",mode = "mean"
)

modl.plot_oos(
    y_true_tf[1], y_pred_tf[1], dates=dates_tf[1],
    title=f"TimesFM OOS (R²_OS={r2_tf[1]:.4f})",
    ylabel="Monthly equity premium", mode = "mean"
)


In [ ]:
r2_m, y_true_m, y_pred_m, dates_m = modl.moirai2_oos(
    data, covariates=(), start_oos="1965-01-01",
    ctx=240, device="cpu", ct_cutoff=True, quiet=False, mode = "mean"
)

modl.plot_oos(
    y_true_m[1], y_pred_m[1], dates=dates_m[1],
    title=f"Moirai-2 OOS (R²_OS={r2_m[1]:.4f})",
    ylabel="Monthly equity premium", mode = "mean"
)

In [ ]:
r2_m, y_true_m, y_pred_m, dates_m = modl.moirai2_oos(
    data, covariates=("d/p","tms","dfy"), start_oos="1965-01-01",
    ctx=240, device="cpu", ct_cutoff=True, quiet=False, mode = "mean"
)

modl.plot_oos(
    y_true_m[1], y_pred_m[1], dates=dates_m[1],
    title=f"Moirai-2 OOS (R²_OS={r2_m[1]:.4f})",
    ylabel="Monthly equity premium", mode = "mean"
)

In [ ]:


r2, y_true, y_pred, dates_fs = modl.flowstate_oos(
    data, mode = "mean", ct_cutoff=True
)

In [ ]:
modl.plot_oos(
    y_true[1],
    y_pred[1],
    dates=dates_fs[1],
    title=f"FlowState OOS (R²_OS={r2[1]:.4f})",
    ylabel="Monthly equity premium",
    mode = "mean"
)

In [21]:
from importlib import reload
reload(modl)

<module 'source.regression_models' from 'c:\\Users\\kilia\\Masterarbeit\\source\\regression_models.py'>

In [ ]:
r2_tabpfn, y_true_tabpfn, y_pred_tabpfn, dates_tabpfn = modl.tabpfn_oos_fit_each_step(
    data,
    variables=("d/p", "tms", "dfy"),
    start_oos="1965-01-01",
    lag=1,
    min_train=120,
    ct_cutoff=True,
    quiet=False,
    model_params='2.5',  # or just omit this argument,
    )




In [31]:
from importlib import reload
reload(modl)

<module 'source.regression_models' from 'c:\\Users\\kilia\\Masterarbeit\\source\\regression_models.py'>

In [32]:
r2, y_true, y_pred, dates = modl.tabpfn_advanced_ts_oos(
    data,
    variables=("d/p", "tms", "dfy"),
    target_col="equity_premium",
    lag=3,                  # Use t-1, t-2, t-3
    autoregressive=True,    # Use past returns
    start_oos="1965-01-01",
    model_params = "121"
)

1965-01-01 00:00:00
1965-02-01 00:00:00
1965-03-01 00:00:00
1965-04-01 00:00:00
1965-05-01 00:00:00
1965-06-01 00:00:00
1965-07-01 00:00:00
1965-08-01 00:00:00
1965-09-01 00:00:00
1965-10-01 00:00:00
1965-11-01 00:00:00
1965-12-01 00:00:00
1966-01-01 00:00:00
1966-02-01 00:00:00
1966-03-01 00:00:00
1966-04-01 00:00:00
1966-05-01 00:00:00
1966-06-01 00:00:00
1966-07-01 00:00:00
1966-08-01 00:00:00
1966-09-01 00:00:00
1966-10-01 00:00:00
1966-11-01 00:00:00
1966-12-01 00:00:00
1967-01-01 00:00:00
1967-02-01 00:00:00
1967-03-01 00:00:00
1967-04-01 00:00:00
1967-05-01 00:00:00
1967-06-01 00:00:00
1967-07-01 00:00:00
1967-08-01 00:00:00
1967-09-01 00:00:00
1967-10-01 00:00:00
1967-11-01 00:00:00
1967-12-01 00:00:00
1968-01-01 00:00:00
1968-02-01 00:00:00
1968-03-01 00:00:00
1968-04-01 00:00:00
1968-05-01 00:00:00
1968-06-01 00:00:00
1968-07-01 00:00:00
1968-08-01 00:00:00
1968-09-01 00:00:00
1968-10-01 00:00:00
1968-11-01 00:00:00
1968-12-01 00:00:00
1969-01-01 00:00:00
1969-02-01 00:00:00


In [24]:
r2_ts

(-0.014455372020061974,
 {'mean': -0.013821511580861773,
  'std': 0.007390493221755679,
  'lower': -0.02755925454731974,
  'upper': 0.0006980707120551393})

In [ ]:
modl.plot_oos(
    y_true_tabpfn, y_pred_tabpfn, dates=dates_tabpfn,
    title=f"TabPFN (fit each step) OOS (R²_OS={r2_tabpfn:.4f})",
    ylabel="Monthly equity premium"
)

In [ ]:
r2_pfn_ts, y_true_pfn_ts, y_pred_pfn_ts, dates_pfn_ts = modl.tabpfn_ts_oos_fit_each_step(
    data,
    target_col="equity_premium",
    start_oos="1965-01-01",
    ctx=240,
    freq="M",
    min_windows=120,
    ct_cutoff=False,
    quiet=False,
    fit_kwargs={"epochs": 1}  # if your build supports epochs
)

plot_oos(
    y_true_pfn_ts, y_pred_pfn_ts, dates=dates_pfn_ts,
    title=f"TabPFN-TS (fit each step) OOS (R²_OS={r2_pfn_ts:.4f})",
    ylabel="Monthly equity premium"
)


In [ ]:
!pip install tabpfn-ts

In [ ]:
# Example usage
results = {
    "OLS(d/p)": (r2_o, y_true_o, y_pred_o, dates_o),
    "Chronos": (r2_c[1], y_true_c[1], y_pred_c[1], dates_c[1]),
    "TimesFM": (r2_tf[1], y_true_tf[1], y_pred_tf[1], dates_tf[1]),
}
modl.plot_oos_multi(results)

In [ ]:
from importlib import reload
reload(mm)
import source.modelling_multistep as mm
# 12-step multi-horizon OOS with Chronos
r2_c, y_true_c, y_pred_c, dates_c = mm.chronos_oos_multistep(
    data,
    start_oos="1965-01-01",
    prediction_length=100,
    ct_cutoff=False,
    quiet=False,
    freq="M",
)

# Check R² by horizon
for h in range(1, 13):
    print(f"h={h}: R²_OS = {r2_c[h]:.4f}")

# Plot horizon-1 forecast
h = 1
modl.plot_oos(
    y_true_c[h], y_pred_c[h], dates=dates_c[h],
    title=f"Chronos-Bolt OOS (h={h}, R²_OS={r2_c[h]:.4f})",
    ylabel="Monthly equity premium",
)

# Plot horizon-12 forecast
h = 100
modl.plot_oos(
    y_true_c[h], y_pred_c[h], dates=dates_c[h],
    title=f"Chronos-Bolt OOS (h={h}, R²_OS={r2_c[h]:.4f})",
    ylabel="Monthly equity premium",
)


In [ ]:
from importlib import reload
reload(modl)    
import warnings
warnings.filterwarnings('ignore')  # Suppress all warnings

r2_auto, trues_auto, preds_auto, dates_auto = modl.autoarima_oos(
    data=data,
    target_col="equity_premium",
    start_oos="1970-01-01",
    freq="M",
    min_history_months=60,
    auto_arima_kwargs={"max_p": 3, "max_q": 3},
    order_search_every=15,   # search only once (first step)
    refit_each_step=True,      # <-- keep refitting model each step
    mode = "rw",
)


In [ ]:
base_cols=["d/p", "d/y" ]
base_cols = ["d/p", "d/y" , "e/p", "d/e","svar", "b/m", "ntis","tbl", "ltr", "tms", "dfy", "dfr", "infl"]
fit_fn = modl.make_mlp_lag_reg_fit_predict_fn(
    base_cols=base_cols, n_lags=24, epochs=30, retrain_every=10, print_loss=True
)


r2, y_true, y_pred, dates = modl.expanding_oos_tabular(
    data=data, target_col="equity_premium",
    feature_cols=None,  # ignored by driver when using fit_predict
    start_oos="1965-01-01", start_date="1927-01-01",
    min_train=120, model_name="MLPReg(lags)",
    model_fit_predict_fn=fit_fn, mode="mean"
)
print(f"MLPReg(lags) OOS R²: {r2:.4f}")


In [ ]:
print(f"MLPReg(lags) OOS R²: {r2:.4f}")
print("Ha")

In [ ]:
data.columns

In [ ]:
from importlib import reload    
reload(modl)

In [ ]:
fit_fn = modl.make_lstm_seq_reg_fit_predict_fn(
    feature_cols=["d/p", "d/y"],  # or whatever features you want
    seq_len=24, epochs=20, retrain_every=5
)

r2, y_true, y_pred, dates = modl.expanding_oos_tabular(
    data=data, target_col="equity_premium",
    feature_cols=None,
    start_oos="1965-01-01", start_date="1927-01-01",
    min_train=120, model_name="LSTMReg(seq)",
    model_fit_predict_fn=fit_fn, mode="mean"
)


In [ ]:
fit_fn = modl.make_transformer_seq_reg_fit_predict_fn(
    feature_cols=["d/p", "tms", "dfy"],
    target_col="equity_premium",
    seq_len=24,           # 12–36 are good starting points
    epochs=20,
    retrain_every=5,     # periodic retraining
    d_model=128, nhead=4, num_layers=2,
)

r2, y_true, y_pred, dates = modl.expanding_oos_tabular(
    data=data,
    target_col="equity_premium",
    feature_cols=None,               # ignored when using fit_predict closure
    start_oos="1965-01-01",
    start_date="1927-01-01",
    min_train=120,
    model_name="TransformerReg(seq)",
    model_fit_predict_fn=fit_fn,
    mode="mean",
)
